In [7]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GRU, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# ------------------------------
# 1. 数据读取与基本预处理
# ------------------------------
file_path = 'IMPACT.sensors.csv'
data = pd.read_csv(file_path)

# 确保有时间戳列并转换为 datetime 类型
timestamp_column = 'createdAt'
data[timestamp_column] = pd.to_datetime(data[timestamp_column])

# 按时间排序
data = data.sort_values(by=timestamp_column).reset_index(drop=True)

# 简单检查数据缺失情况（可根据需要做插值或去除）
print("缺失情况：")
print(data.isnull().sum())

# 演示插值：若缺失较少或轻微，可以用以下方式
# data = data.interpolate(method='time')  # 基于时间序列插值

# ------------------------------
# 2. 特征选择与生成
#    - 在 pH 基础上添加温度 temperature
#    - 构造时间特征 hour, day_of_week, is_weekend
# ------------------------------
# 可以根据实际传感器数据列名进行调整
features = ['pH', 'temperature']  # 如果只想加温度，可保留这两个；有更多传感器，可一并加入

# 提取必要列
data_features = data[features].copy()

# 生成时间特征
data['hour'] = data[timestamp_column].dt.hour
data['day_of_week'] = data[timestamp_column].dt.dayofweek
data['is_weekend'] = data['day_of_week'].isin([5, 6]).astype(int)

# 将时间特征也加入到特征列表中
time_features = ['hour', 'day_of_week', 'is_weekend']
data_features[time_features] = data[time_features]

# 查看特征相关性（可根据相关性筛选特征）
corr_matrix = data_features.corr()
print("\n特征相关性：")
print(corr_matrix)

# 选取最终特征
final_features = features + time_features

# ------------------------------
# 3. 数据归一化
# ------------------------------
scaler = MinMaxScaler()
data_normalized = scaler.fit_transform(data_features[final_features])  # 按列归一化

# ------------------------------
# 4. 构建序列数据（含滑窗步长）
#    - sequence_length: 过去多少个时间步
#    - prediction_steps: 要预测哪些未来点
#    - step_size: 滑窗移动步长（原为1，这里设6，可减少样本相关性）
# ------------------------------

sequence_length = 12  # 过去 2 个时间步（可视为 12*10min = 120min，例如）
prediction_steps = [6, 12, 18, 24, 30, 36, 42, 48]  # 预测未来 1~8 小时(以10min为间隔)
step_size = 6  # 滑窗步长

def create_future_sequences(data_array, sequence_length, prediction_steps, step_size=6):
    X, y = [], []
    max_step = max(prediction_steps)

    for i in range(0, len(data_array) - sequence_length - max_step, step_size):
        # 取过去 sequence_length 长度的数据作为X
        X.append(data_array[i : i + sequence_length])

        # 对于每个预测步长，取后续几个点的平均（或其他统计方式）作为标签
        future_values = [
            np.mean(data_array[i + sequence_length + p : i + sequence_length + p + 6])
            for p in prediction_steps
        ]
        y.append(future_values)

    return np.array(X), np.array(y)

X, y = create_future_sequences(data_normalized, sequence_length, prediction_steps, step_size)

print(f"\n构造后的 X.shape = {X.shape}, y.shape = {y.shape}")

# ------------------------------
# 5. 划分训练集和测试集
# ------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False  # 时间序列通常不建议打乱
)

print(f"训练集大小: {X_train.shape}, 测试集大小: {X_test.shape}")

# ------------------------------
# 6. 构建增强型 GRU 网络
#    - 3 层 GRU + Dropout + BatchNorm
# ------------------------------
sequence_input = Input(shape=(sequence_length, len(final_features)))

# 第1层 GRU
x = GRU(128, activation='tanh', return_sequences=True)(sequence_input)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)

# 第2层 GRU
x = GRU(64, activation='tanh', return_sequences=True)(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)

# 第3层 GRU
x = GRU(32, activation='tanh')(x)
x = BatchNormalization()(x)

# 全连接层
x = Dense(64, activation='swish')(x)
x = Dropout(0.2)(x)

# 输出层 - 预测多个时间步
output = Dense(len(prediction_steps), activation='linear')(x)

model = Model(inputs=sequence_input, outputs=output)
model.compile(optimizer='adam', loss='mse')

model.summary()

# ------------------------------
# 7. 训练策略优化 - 动态学习率 & 早停
# ------------------------------
lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-6
)

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

# ------------------------------
# 8. 训练模型
# ------------------------------
history = model.fit(
    X_train, y_train,
    epochs=42,
    batch_size=32,
    validation_data=(X_test, y_test),
    callbacks=[lr_scheduler, early_stop],
    verbose=1
)

# ------------------------------
# 9. 测试集评估
# ------------------------------
test_loss = model.evaluate(X_test, y_test, verbose=0)
print("\nTest Loss (MSE):", test_loss)

y_pred = model.predict(X_test)

# 逐步评估每个预测步
def evaluate_predictions(y_true, y_pred, steps):
    print("\nStep-wise Performance Evaluation:")
    for idx, step in enumerate(steps):
        # 假设每个数据点间隔为10分钟
        hours = (step * 10) // 60
        minutes = (step * 10) % 60

        true_step = y_true[:, idx]
        pred_step = y_pred[:, idx]

        mae = mean_absolute_error(true_step, pred_step)
        r2 = r2_score(true_step, pred_step)
        print(f"预测 {hours}h{minutes:02d}min 后 -> MAE={mae:.4f}, R²={r2:.4f}")

    # 整体评估
    overall_mae = mean_absolute_error(y_true, y_pred)
    overall_r2 = r2_score(y_true.flatten(), y_pred.flatten())
    print("\nOverall Performance:")
    print(f"Overall MAE: {overall_mae:.4f}")
    print(f"Overall R²: {overall_r2:.4f}")

evaluate_predictions(y_test, y_pred, prediction_steps)

# ------------------------------
# 10. 模型保存
# ------------------------------
model.save('ph_prediction_gru_model.keras')


缺失情况：
_id             0
createdAt       0
temperature     0
pH              0
conductivity    0
oxygen          0
ppm             0
waterLevel      0
pm25            0
__v             0
dtype: int64

特征相关性：
                   pH  temperature      hour  day_of_week  is_weekend
pH           1.000000    -0.255652 -0.029854     0.044361    0.022740
temperature -0.255652     1.000000 -0.000693     0.028069    0.048097
hour        -0.029854    -0.000693  1.000000    -0.003893   -0.004632
day_of_week  0.044361     0.028069 -0.003893     1.000000    0.787225
is_weekend   0.022740     0.048097 -0.004632     0.787225    1.000000

构造后的 X.shape = (3273, 12, 5), y.shape = (3273, 8)
训练集大小: (2618, 12, 5), 测试集大小: (655, 12, 5)
Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 12, 5)]           0         
                                                                 